In [23]:
import pandas as pd
df_news = pd.read_csv('data_final2.csv')
drop_columns=['Date','Author(s)','Tags','URL']
df_news = df_news.drop(columns=drop_columns)
df_news.head(10)
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df_news['label_id'] = label_encoder.fit_transform(df_news['Category'])
df_news[['Contents','Category']].to_csv('data_news.csv',index=False)
df_news=pd.read_csv('data_news.csv')

In [24]:
import pandas as pd
import numpy as np
import pickle
import regex as re
from underthesea import word_tokenize

uniChars = "àáảãạâầấẩẫậăằắẳẵặèéẻẽẹêềếểễệđìíỉĩịòóỏõọôồốổỗộơờớởỡợùúủũụưừứửữựỳýỷỹỵÀÁẢÃẠÂẦẤẨẪẬĂẰẮẲẴẶÈÉẺẼẸÊỀẾỂỄỆĐÌÍỈĨỊÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢÙÚỦŨỤƯỪỨỬỮỰỲÝỶỸỴÂĂĐÔƠƯ"
unsignChars = "aaaaaaaaaaaaaaaaaeeeeeeeeeeediiiiiooooooooooooooooouuuuuuuuuuuyyyyyAAAAAAAAAAAAAAAAAEEEEEEEEEEEDIIIOOOOOOOOOOOOOOOOOOOUUUUUUUUUUUYYYYYAADOOU"

def loaddicchar():
    dic = {}
    char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split(
        '|')
    charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ".split(
        '|')
    for i in range(len(char1252)):
        dic[char1252[i]] = charutf8[i]
    return dic
dicchar = loaddicchar()

# Hàm chuyển Unicode dựng sẵn về Unicde tổ hợp (phổ biến hơn)
def convert_unicode(txt):
    return re.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dicchar[x.group()], txt)

bang_nguyen_am = [['a', 'à', 'á', 'ả', 'ã', 'ạ', 'a'],
                  ['ă', 'ằ', 'ắ', 'ẳ', 'ẵ', 'ặ', 'aw'],
                  ['â', 'ầ', 'ấ', 'ẩ', 'ẫ', 'ậ', 'aa'],
                  ['e', 'è', 'é', 'ẻ', 'ẽ', 'ẹ', 'e'],
                  ['ê', 'ề', 'ế', 'ể', 'ễ', 'ệ', 'ee'],
                  ['i', 'ì', 'í', 'ỉ', 'ĩ', 'ị', 'i'],
                  ['o', 'ò', 'ó', 'ỏ', 'õ', 'ọ', 'o'],
                  ['ô', 'ồ', 'ố', 'ổ', 'ỗ', 'ộ', 'oo'],
                  ['ơ', 'ờ', 'ớ', 'ở', 'ỡ', 'ợ', 'ow'],
                  ['u', 'ù', 'ú', 'ủ', 'ũ', 'ụ', 'u'],
                  ['ư', 'ừ', 'ứ', 'ử', 'ữ', 'ự', 'uw'],
                  ['y', 'ỳ', 'ý', 'ỷ', 'ỹ', 'ỵ', 'y']]
bang_ky_tu_dau = ['', 'f', 's', 'r', 'x', 'j']

nguyen_am_to_ids = {}

for i in range(len(bang_nguyen_am)):
    for j in range(len(bang_nguyen_am[i]) - 1):
        nguyen_am_to_ids[bang_nguyen_am[i][j]] = (i, j)

def chuan_hoa_dau_tu_tieng_viet(word):
    if not is_valid_vietnam_word(word):
        return word

    chars = list(word)
    dau_cau = 0
    nguyen_am_index = []
    qu_or_gi = False
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x == -1:
            continue
        elif x == 9:  # check qu
            if index != 0 and chars[index - 1] == 'q':
                chars[index] = 'u'
                qu_or_gi = True
        elif x == 5:  # check gi
            if index != 0 and chars[index - 1] == 'g':
                chars[index] = 'i'
                qu_or_gi = True
        if y != 0:
            dau_cau = y
            chars[index] = bang_nguyen_am[x][0]
        if not qu_or_gi or index != 1:
            nguyen_am_index.append(index)
    if len(nguyen_am_index) < 2:
        if qu_or_gi:
            if len(chars) == 2:
                x, y = nguyen_am_to_ids.get(chars[1])
                chars[1] = bang_nguyen_am[x][dau_cau]
            else:
                x, y = nguyen_am_to_ids.get(chars[2], (-1, -1))
                if x != -1:
                    chars[2] = bang_nguyen_am[x][dau_cau]
                else:
                    chars[1] = bang_nguyen_am[5][dau_cau] if chars[1] == 'i' else bang_nguyen_am[9][dau_cau]
            return ''.join(chars)
        return word

    for index in nguyen_am_index:
        x, y = nguyen_am_to_ids[chars[index]]
        if x == 4 or x == 8:  # ê, ơ
            chars[index] = bang_nguyen_am[x][dau_cau]
            # for index2 in nguyen_am_index:
            #     if index2 != index:
            #         x, y = nguyen_am_to_ids[chars[index]]
            #         chars[index2] = bang_nguyen_am[x][0]
            return ''.join(chars)

    if len(nguyen_am_index) == 2:
        if nguyen_am_index[-1] == len(chars) - 1:
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
            chars[nguyen_am_index[0]] = bang_nguyen_am[x][dau_cau]
            # x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
            # chars[nguyen_am_index[1]] = bang_nguyen_am[x][0]
        else:
            # x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
            # chars[nguyen_am_index[0]] = bang_nguyen_am[x][0]
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
            chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
    else:
        # x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
        # chars[nguyen_am_index[0]] = bang_nguyen_am[x][0]
        x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
        chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
        # x, y = nguyen_am_to_ids[chars[nguyen_am_index[2]]]
        # chars[nguyen_am_index[2]] = bang_nguyen_am[x][0]
    return ''.join(chars)

def is_valid_vietnam_word(word):
    chars = list(word)
    nguyen_am_index = -1
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x != -1:
            if nguyen_am_index == -1:
                nguyen_am_index = index
            else:
                if index - nguyen_am_index != 1:
                    return False
                nguyen_am_index = index
    return True

def chuan_hoa_dau_cau_tieng_viet(sentence):
    """
        Chuyển câu tiếng việt về chuẩn gõ dấu kiểu cũ.
        :param sentence:
        :return:
        """
    sentence = sentence.lower()
    words = sentence.split()
    for index, word in enumerate(words):
        cw = re.sub(r'(^\p{P}*)([p{L}.]*\p{L}+)(\p{P}*$)', r'\1/\2/\3', word).split('/')
        # print(cw)
        if len(cw) == 3:
            cw[1] = chuan_hoa_dau_tu_tieng_viet(cw[1])
        words[index] = ''.join(cw)
    return ' '.join(words)

stopword = set()
file = r'data/vietnamese-stopwords.txt'
for line in open(file, "r",encoding="utf8"):
  line = line.strip('\n')
  stopword.add(line)

def remove_stopwords(line):
    words = []
    for word in line.strip().split():
        if word not in stopword:
            words.append(word)
    return ' '.join(words)

def text_preprocess(document):
    # chuẩn hóa unicode
    document = convert_unicode(document)
    # chuẩn hóa cách gõ dấu tiếng Việt
    document = chuan_hoa_dau_cau_tieng_viet(document)
    # tách từ
    document = word_tokenize(document, format="text")
    # xóa stopwords
    document = remove_stopwords(document)
    # đưa về lower
    document = document.lower()
    # xóa các ký tự không cần thiết
    document = re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ_]',' ',document)
    # xóa khoảng trắng thừa
    document = re.sub(r'\s+', ' ', document).strip()
    return document

In [25]:
# text='''Một tiếng trước khi phái đoàn Việt Nam đến, đoàn kỵ binh và đội danh dự của Hungary diễn tập lễ đón tại Quảng trường Nhà Quốc hội dưới cơn mưa rả rích và cái lạnh 5 độ C. Tuy nhiên chỉ khoảng 10 phút trước khi lễ đón chính thức diễn ra, trời tạnh mưa và hửng nắng.
# 
# Giống như nghi thức đón tiếp các lãnh đạo cấp cao nước ngoài đến thăm Hungary, lễ đón Thủ tướng Phạm Minh Chính diễn ra dưới sự chứng kiến trực tiếp của người dân và du khách ở trung tâm Budapest.
# 
# Khoảng 14h30 theo giờ địa phương, đội kỵ binh danh dự Ferenc Nádsdy tháp tùng xe của Thủ tướng Phạm Minh Chính vào đến vị trí danh dự, nơi Thủ tướng Viktor Orbán đã đứng đợi nhiều phút trước đó.'''
# document = chuan_hoa_dau_cau_tieng_viet(text)
# document=word_tokenize(document, format="text")
# document = remove_stopwords(document)
# print(document)
# print("----")
# document = document.lower() # convert to lowercase
# document = re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ_]',' ',document) # remove punctuation
# document = re.sub(r'\s+', ' ', document).strip() # remove whitespace
# print(document)
#df_news['Title'] = df_news['Title'].apply(lambda x : text_preprocess(x))
df_news['Contents'] = df_news['Contents'].apply(lambda x : text_preprocess(x))
#df_news['text'] = df_news['Title'] + ' ' + df_news['Contents']
df_news['text'] = df_news['Contents']

In [26]:
df_news = df_news.reset_index(drop=True)
df_news

,Contents,Category,text
0,bđs nội_đô hấp_dẫn hạ_tầng thủ_đô hà_nội phát_...,Bất động sản,bđs nội_đô hấp_dẫn hạ_tầng thủ_đô hà_nội phát_...
1,ubnd tỉnh phú_yên văn_bản giao_sở xây_dựng chủ...,Bất động sản,ubnd tỉnh phú_yên văn_bản giao_sở xây_dựng chủ...
2,hàng_loạt thông_tin báo_chí phản_ánh một_số dự...,Bất động sản,hàng_loạt thông_tin báo_chí phản_ánh một_số dự...
3,4 đường 3 đường giấy quảng_cáo dự_án the eden ...,Bất động sản,4 đường 3 đường giấy quảng_cáo dự_án the eden ...
4,sở tài_nguyên môi_trường đề_nghị ubnd tỉnh báo...,Bất động sản,sở tài_nguyên môi_trường đề_nghị ubnd tỉnh báo...
...,...,...,...
2409,mới_đây phát_hiện mắc bệnh không_thể tiếp_tục ...,Tin hoạt động,mới_đây phát_hiện mắc bệnh không_thể tiếp_tục ...
2410,chương_trình phát đi hình_ảnh cuộc_sống bé vùn...,Tin hoạt động,chương_trình phát đi hình_ảnh cuộc_sống bé vùn...
2411,liên bỏng toàn_thân đau_đớn ngày_nay tình_trạn...,Tin hoạt động,liên bỏng toàn_thân đau_đớn ngày_nay tình_trạn...
2412,đầu_tiên khánh_thành bàn_giao 2 tài_trợ sym vi...,Tin hoạt động,đầu_tiên khánh_thành bàn_giao 2 tài_trợ sym vi...


In [27]:
total_label = 9
# Lấy túi từ vựng của dữ liệu đầu vào
vocab = {}
label_vocab = {}
for idx, row in  df_news.iterrows():
    words = row['text'].split()
    label = row['Category']
    if label not in label_vocab:
        label_vocab[label] = {}
    for word in words:
        label_vocab[label][word] = label_vocab[label].get(word, 0) + 1
        if word not in vocab:
            vocab[word] = set()
        vocab[word].add(label)

In [28]:
print(len(vocab))
print(vocab)

37281
{'bđs': {'Lao Động & Đời sống', 'Lao Động cuối tuần', 'Bất động sản', 'Phóng sự', 'Lưu trữ', 'Kinh doanh'}, 'nội_đô': {'Lao Động & Đời sống', 'Xe +', 'Xã hội', 'Gia đình - Hôn nhân', 'Bất động sản', 'Phóng sự', 'Thời sự'}, 'hấp_dẫn': {'Lao Động & Đời sống', 'Sức khỏe', 'Thể thao', 'Giáo dục', 'Xe +', 'Xã hội', 'Tin tức việc làm', 'Văn hóa - Giải trí', 'Gia đình - Hôn nhân', 'Diễn đàn', 'Lao Động cuối tuần', 'Bất động sản', 'Bạn đọc', 'Lưu trữ', 'Thời sự', 'Công nghệ', 'Kinh doanh'}, 'hạ_tầng': {'Lao Động & Đời sống', 'Pháp luật', 'Sự kiện Bình luận', 'Giáo dục', 'Xe +', 'Xã hội', 'Thông tin tiện ích', 'Lao Động cuối tuần', 'Bất động sản', 'Phóng sự', 'Bạn đọc', 'Lưu trữ', 'Thời sự', 'Công nghệ', 'Tin hoạt động', 'Kinh doanh'}, 'thủ_đô': {'Sức khỏe', 'Sự kiện Bình luận', 'Giáo dục', 'Tin tức việc làm', 'Xã hội', 'Thế giới', 'Công đoàn', 'Văn hóa - Giải trí', 'Lao Động cuối tuần', 'Bất động sản', 'Phóng sự', 'Bạn đọc', 'Lưu trữ', 'Thời sự', 'Công nghệ', 'Tin hoạt động', 'Kinh doanh

In [29]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import np_utils
test_percent = 0.2
from tensorflow.keras.utils import to_categorical



In [30]:
X_train, X_test, y_train, y_test = train_test_split(df_news['text'], df_news['Category'], test_size=test_percent, random_state=42)
unique_classes_train = np.unique(y_train)
unique_classes_test = np.unique(y_test)

print("Unique classes in Y_train:", unique_classes_train)
print("Unique classes in Y_test:", unique_classes_test)
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
label_encoder.fit(y_test)
print(list(label_encoder.classes_), '\n')
Y_train = label_encoder.transform(y_train)
Y_test = label_encoder.transform(y_test)
num_classes = len(np.unique(Y_train))  # Calculate the actual number of classes
Y_train = to_categorical(Y_train, num_classes)
Y_test = to_categorical(Y_test, num_classes)

Unique classes in Y_train: ['Bạn đọc' 'Bất động sản' 'Công nghệ' 'Công đoàn' 'Diễn đàn'
 'Gia đình - Hôn nhân' 'Giáo dục' 'Kinh doanh' 'Lao Động & Đời sống'
 'Lao Động cuối tuần' 'Lưu trữ' 'Pháp luật' 'Phóng sự' 'Sức khỏe'
 'Sự kiện Bình luận' 'Thông tin tiện ích' 'Thế giới' 'Thể thao' 'Thời sự'
 'Tin hoạt động' 'Tin tức việc làm' 'Tấm Lòng Vàng' 'Văn hóa - Giải trí'
 'Xe +' 'Xã hội']
Unique classes in Y_test: ['Bạn đọc' 'Bất động sản' 'Công nghệ' 'Công đoàn' 'Diễn đàn'
 'Gia đình - Hôn nhân' 'Giáo dục' 'Kinh doanh' 'Lao Động & Đời sống'
 'Lao Động cuối tuần' 'Lưu trữ' 'Pháp luật' 'Phóng sự' 'Sức khỏe'
 'Sự kiện Bình luận' 'Thông tin tiện ích' 'Thế giới' 'Thể thao' 'Thời sự'
 'Tin hoạt động' 'Tin tức việc làm' 'Tấm Lòng Vàng' 'Văn hóa - Giải trí'
 'Xe +' 'Xã hội']
['Bạn đọc', 'Bất động sản', 'Công nghệ', 'Công đoàn', 'Diễn đàn', 'Gia đình - Hôn nhân', 'Giáo dục', 'Kinh doanh', 'Lao Động & Đời sống', 'Lao Động cuối tuần', 'Lưu trữ', 'Pháp luật', 'Phóng sự', 'Sức khỏe', 'Sự kiện Bình luậ

In [31]:
print(X_train)

831     doanh_nghiệp đầu_tư dự_án du_lịch nam ô resort...
2381    7 cháu nguyễn_ngọc_quý 12 trú xóm 11 xã nam_tr...
605     nườm_nượp xe_cộ ra_vào bãi trông_giữ phép báo ...
2412    đầu_tiên khánh_thành bàn_giao 2 tài_trợ sym vi...
599     trao_sổ tiết_kiệm công_đoàn sổ trị_giá 10 triệ...
                              ...                        
1638    tân huấn_luyện_viên ronald koeman đánh_giá tài...
1095    phim canh_bạc tình_yêu tập 92 thanh_vân chết k...
1130    trưa hôm_nay 22 9 nghệ_sĩ việt hoài_linh vy_oa...
1294    hiệu_trưởng kiêm cò thầy n t h giáo_viên hợp_đ...
860     oái_oăm dư_luận tốn bao hơi sức trong_sạch lò_...
Name: text, Length: 1931, dtype: object


In [38]:
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

max_words = len(vocab.keys())

tok = Tokenizer(num_words = max_words)
tok.fit_on_texts(X_train)
pickle.dump(tok,open('tokenize_text.pkl', 'wb'))

sequences = tok.texts_to_sequences(X_train)
max_len = 2000

sequences_matrix = pad_sequences(sequences,maxlen = max_len,padding='post')

# In tất cả các số trong sequences_matrix câu thứ nhất
print('Câu thứ nhất:', X_train[1])
print('Label câu thứ nhất:', y_train[1])
print('Độ dài: ', len(sequences[1]))
print('Tổng số từ', len(sequences))
print('------------Sau khi tokenize-------------------------')
print('Sequences_matrix shape: ', sequences_matrix.shape)

# In sequences_matrix câu thứ nhất
print('Sequences_matrix câu thứ nhất:')
for num in sequences_matrix[1, :]:
    print(num, end=' ')


Câu thứ nhất: ubnd tỉnh phú_yên văn_bản giao_sở xây_dựng chủ_trì phối_hợp ngành chức_năng liên_quan lập báo_cáo nghiên_cứu tiền_khả_thi dự_án xây_dựng tập_trung trụ_sở cơ_quan nhà_nước tỉnh phú_yên lưu_ý sử_dụng đất vàng trình thủ_tướng chính_phủ xem_xét quyết_định vừa_qua sở xây_dựng phú_yên tổ_chức hội_nghị ý_kiến dự_thảo đề_án xây_dựng tập_trung trụ_sở cơ_quan nhà_nước tỉnh mục_tiêu đề_án sắp_xếp trụ_sở làm_việc cơ_quan_hành_chính tỉnh hiện nằm phân_tán khu_vực tp hòa đơn_vị thực_hiện đề_án xác_định giá_trị tài_sản cơ_quan nhà_nước hiện dự_kiến vốn thu quyền sử_dụng đất tài_sản đất cân_đối thu_chi xây_dựng khu hành_chính tổng đầu_tư dự_án xây_dựng tập_trung trụ_sở cơ_quan nhà_nước tỉnh phú_yên dự_kiến 473 tỉ đồng giá_trị đất tài_sản thanh_lý cơ_quan hiện_tại dự_kiến di_dời 3 477 tỉ đồng
Label câu thứ nhất: Bất động sản
Độ dài:  313
Tổng số từ 1931
------------Sau khi tokenize-------------------------
Sequences_matrix shape:  (1931, 2000)
Sequences_matrix câu thứ nhất:
226 455 107 53